In [1]:
import tensorflow as tf

In [2]:
class Model:
    def __init__(self, learning_rate):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([660726,40]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.logits = tf.layers.dense(embed, 4)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = self.logits, labels = self.Y))

In [3]:
sess = tf.InteractiveSession()
model = Model(1e-4)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'lang-detection-w/model.ckpt')

INFO:tensorflow:Restoring parameters from lang-detection-w/model.ckpt


In [4]:
import pickle
with open('language-detection-vectorizer.pkl','rb') as fopen:
    bow_chars = pickle.load(fopen)

In [5]:
chinese_text = '今天是６月１８号，也是Muiriel的生日！'
english_text = 'i totally love it man'
indon_text = 'menjabat saleh perombakan menjabat periode komisi energi fraksi partai pengurus partai periode periode partai terpilih periode menjabat komisi perdagangan investasi persatuan periode'
malay_text = 'beliau berkata program Inisitif Peduli Rakyat (IPR) yang diperkenalkan oleh kerajaan negeri Selangor lebih besar sumbangannya'

In [6]:
import numpy as np
def convert_sparse_matrix_to_sparse_tensor(X, limit = 5):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    coo.data[coo.data > limit] = limit
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [7]:
transformed = bow_chars.transform([chinese_text,english_text,indon_text,malay_text])
batch_x = convert_sparse_matrix_to_sparse_tensor(transformed)
np.argmax(sess.run(model.logits, feed_dict = {model.X: batch_x[0], model.W: batch_x[1]}),axis=1)

array([0, 1, 2, 3])